In [1]:
# Near field
# 
import meep as mp
import numpy as np

# --- Simulation parameters ---
wavelength = 1 # μm
frequency = 1 / wavelength  # Meep units: c = 1 # μm
resolution = 10  # pixels/μm

pml_thickness = 1.0  # μm
padding = 2.0        # distance from source to PML
cell_dim = 2 * (padding + pml_thickness)
cell_size = mp.Vector3(cell_dim, cell_dim, cell_dim)

# Define direction vector (unit normalized)
d = np.array([1, 2, 3]) / np.sqrt(14)

# Source properties
length = 0.325
source_center = mp.Vector3(length,length,length)

sources = [
    mp.Source(mp.ContinuousSource(frequency=frequency),
              component=mp.Ex,
              center=source_center,
              amplitude=d[0]),
    mp.Source(mp.ContinuousSource(frequency=frequency),
              component=mp.Ey,
              center=source_center,
              amplitude=d[1]),
    mp.Source(mp.ContinuousSource(frequency=frequency),
              component=mp.Ez,
              center=source_center,
              amplitude=d[2])
]


def simulate_full_fields(obs_points):
    if obs_points.ndim == 1:
        obs_points = obs_points[np.newaxis, :]

    sim = mp.Simulation(
    cell_size=cell_size,
    resolution=resolution,
    boundary_layers=[mp.PML(pml_thickness)],
    sources=sources  # ✅ Pass the list directly!
    )


    dft_obj = sim.add_dft_fields(
        [mp.Ex, mp.Ey, mp.Ez, mp.Hx, mp.Hy, mp.Hz],
        [frequency],
        center=mp.Vector3(),
        size=cell_size
    )

    optical_cycles = 5
    sim.run(until=optical_cycles * wavelength)



    Ex = sim.get_dft_array(dft_obj, mp.Ex, 0)
    Ey = sim.get_dft_array(dft_obj, mp.Ey, 0)
    Ez = sim.get_dft_array(dft_obj, mp.Ez, 0)
    Hx = sim.get_dft_array(dft_obj, mp.Hx, 0)
    Hy = sim.get_dft_array(dft_obj, mp.Hy, 0)
    Hz = sim.get_dft_array(dft_obj, mp.Hz, 0)

    grid_size = sim.get_array_metadata()  # ✅ this returns (nx, ny, nz)

    delta = 1.0 / resolution
    origin = -0.5 * np.array([cell_dim, cell_dim, cell_dim])

    def to_index(vec):
        return tuple(((vec - origin) / delta).astype(int))

    E_all = []
    H_all = []

    for pt in obs_points:
        idx = to_index(np.array(pt))
        E = np.array([Ex[idx], Ey[idx], Ez[idx]])
        H = np.array([Hx[idx], Hy[idx], Hz[idx]])
        E_all.append(E)
        H_all.append(H)

    return np.array(E_all), np.array(H_all)

# --- Example usage ---
points = np.array([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0]
])

points = np.array([[3*length,2*length,4*length],
                  [-2*length,3*length,1*length],
                  [-1*length,-1*length,-1*length]]
)


E, H = simulate_full_fields(points)

for i, pt in enumerate(points):
    print(f"\nPoint {i} at {pt}:")
    print(f"  E = {E[i]}")
    print(f" |E| = {np.linalg.norm(E[i])}")
    print(f"  H = {H[i]}")
    print(f" |H| = {np.linalg.norm(H[i])}")
    print(f" |E|/|H| = {np.linalg.norm(E[i]) / np.linalg.norm(H[i])}")



-----------
Initializing structure...
time for choose_chunkdivision = 0.00156093 s
Working in 3D dimensions.
Computational cell is 6 x 6 x 6 with resolution 10
time for set_epsilon = 0.321967 s
-----------
run 0 finished at t = 5.0 (100 timesteps)

Point 0 at [0.975 0.65  1.3  ]:
  E = [ 0.08708161+0.00085636j -0.06843668+0.05796507j  0.01403245+0.06646804j]
 |E| = 0.1422757086433324
  H = [ 0.06190416-0.03017211j  0.06369138-0.03495692j -0.0611275 +0.0336887j ]
 |H| = 0.12203507602081566
 |E|/|H| = 1.1658591388844979

Point 1 at [-0.65   0.975  0.325]:
  E = [-0.09866555+0.04615907j -0.13779661+0.07705836j -0.22320379+0.12414779j]
 |E| = 0.3194120396982013
  H = [-0.11780179+0.06976155j -0.19893375+0.10981463j  0.17572791-0.09554955j]
 |H| = 0.33224665766651734
 |E|/|H| = 0.9613702119429643

Point 2 at [-0.325 -0.325 -0.325]:
  E = [ 0.10018554+0.00303078j  0.02158565+0.04069802j -0.05701424+0.07836525j]
 |E| = 0.14683436926953675
  H = [ 0.04645868-0.02315337j -0.09291737+0.04630673j

In [27]:
# Far field

import meep as mp
import numpy as np

def simulate_far_field(far_point):
    sim = mp.Simulation(
        cell_size=cell_size,
        resolution=resolution,
        boundary_layers=[mp.PML(pml_thickness)],
        sources=sources
    )

    # Define a near-field box around the source
    nearfield_box = sim.add_near2far(
        frequency, 0, 1,  # fcen, df, nfreq
        mp.Near2FarRegion(center=mp.Vector3(), size=mp.Vector3(cell_dim - 2 * pml_thickness, 0, 0), direction=mp.X),
        mp.Near2FarRegion(center=mp.Vector3(), size=mp.Vector3(0, cell_dim - 2 * pml_thickness, 0), direction=mp.Y),
        mp.Near2FarRegion(center=mp.Vector3(), size=mp.Vector3(0, 0, cell_dim - 2 * pml_thickness), direction=mp.Z)
    )

    sim.run(until=100)

    # Calculate far-field E and H
    ff_fields = sim.get_farfield(nearfield_box, mp.Vector3(*far_point))
    E = np.array(ff_fields[0:3])
    H = np.array(ff_fields[3:6])

    return E, H

# --- Example usage ---
far_point = [-1, 2, 3]
#far_point = [10,10,10]
E_far, H_far = simulate_far_field(far_point)

print(f"\nFar-field at {far_point} μm:")
print(f"  E = {E_far}")
print(f" |E| = {np.linalg.norm(E_far)}")
print(f"  H = {H_far}")
print(f" |H| = {np.linalg.norm(H_far)}")
print(f" |E|/|H| = {np.linalg.norm(E_far) / np.linalg.norm(H_far)} (should ≈ 376.73 in vacuum)")
print("E and H are not correctly scaled")


-----------
Initializing structure...
time for choose_chunkdivision = 0.00234199 s
Working in 3D dimensions.
Computational cell is 6 x 6 x 6 with resolution 20
time for set_epsilon = 2.44785 s
-----------


: 

In [23]:
E_far

array([-0.76287123-0.01928273j, -0.11705731-0.05073942j,
        0.5287566 -0.08219612j])

In [17]:
# Calculating H using impedence in free space

# Compute radial direction
r_hat = far_point / np.linalg.norm(far_point)


# Impedance of free space
eta0 = 376.730

# Compute far-field H
H_far = np.cross(r_hat, E_far) / eta0

# Display results
print("E_far:", E_far)
print("H_far:", H_far)
print("|E|   =", np.linalg.norm(E_far))
print("|H|   =", np.linalg.norm(H_far))
print("|E|/|H| =", np.linalg.norm(E_far) / np.linalg.norm(H_far))


E_far: [-3.63492958-7.75012171j  0.3715733 -7.59711509j -2.00932693+2.56281466j]
H_far: [-0.00364174+0.01980497j -0.00916158-0.01467624j  0.00489381+0.01638582j]
|E|   = 11.905323915068161
|H|   = 0.031579581469430684
|E|/|H| = 376.99435398130976


In [18]:
import meep as mp
import numpy as np

# --- Simulation Parameters ---
wavelength = 0.325  # μm
frequency = 1 / wavelength
resolution = 10

pml_thickness = 1.0
padding = 2.0
cell_dim = 2 * (padding + pml_thickness)
cell_size = mp.Vector3(cell_dim, cell_dim, cell_dim)

source_pos = mp.Vector3(0, 0, 0)
source_component = mp.Ez

# --- Far-field Simulation Function ---
def simulate_far_field(far_point):
    sim = mp.Simulation(
        cell_size=cell_size,
        resolution=resolution,
        boundary_layers=[mp.PML(pml_thickness)],
        sources=[mp.Source(mp.ContinuousSource(frequency=frequency),
                           component=source_component,
                           center=source_pos)]
    )

    # Near-to-far field monitor surrounding the source
    nearfield_box = sim.add_near2far(
        frequency, 0, 1,  # single frequency
        mp.Near2FarRegion(center=mp.Vector3(), size=mp.Vector3(cell_dim - 2*pml_thickness, 0, 0), direction=mp.X),
        mp.Near2FarRegion(center=mp.Vector3(), size=mp.Vector3(0, cell_dim - 2*pml_thickness, 0), direction=mp.Y),
        mp.Near2FarRegion(center=mp.Vector3(), size=mp.Vector3(0, 0, cell_dim - 2*pml_thickness), direction=mp.Z)
    )

    sim.run(until=100)

    r = mp.Vector3(*far_point)
    ff_fields = sim.get_farfield(nearfield_box, r)

    # Split into E and H (6 values returned per frequency)
    ff_E = np.array(ff_fields[:3])
    ff_H = np.array(ff_fields[3:])
    
    return ff_E, ff_H

# --- Main Execution ---
if __name__ == "__main__":
    far_point = [10.0, 0.0, 0.0]  # μm
    eta0 = 376.730  # Vacuum impedance (Ohm)

    E_far, H_meep = simulate_far_field(far_point)
    r_hat = np.array(far_point) / np.linalg.norm(far_point)
    H_calc = np.cross(r_hat, E_far) / eta0

    # --- Print Results ---
    print(f"\n📍 Far-field at {far_point} μm:")
    print("E_far     =", E_far)
    print("H_meep    =", H_meep)
    print("H_calc    =", H_calc)

    print("\n--- Norms ---")
    print(f"|E|        = {np.linalg.norm(E_far):.6f}")
    print(f"|H_meep|   = {np.linalg.norm(H_meep):.6f}")
    print(f"|H_calc|   = {np.linalg.norm(H_calc):.6f}")

    print("\n--- Ratios ---")
    print(f"|E|/|H_meep|  = {np.linalg.norm(E_far)/np.linalg.norm(H_meep):.6f} (should ≈ η = {eta0})")
    print(f"|E|/|H_calc|  = {np.linalg.norm(E_far)/np.linalg.norm(H_calc):.6f} (should = η)")

    print("\n--- Difference ---")
    print(f"ΔH = ||H_meep - H_calc|| = {np.linalg.norm(H_meep - H_calc):.6e}")


-----------
Initializing structure...
time for choose_chunkdivision = 0.00062108 s
Working in 3D dimensions.
Computational cell is 6 x 6 x 6 with resolution 10


time for set_epsilon = 0.35655 s
-----------
Meep progress: 17.400000000000002/100.0 = 17.4% done in 4.0s, 19.0s to go
on time step 348 (time=17.4), 0.011509 s/step
Meep progress: 41.050000000000004/100.0 = 41.1% done in 8.0s, 11.5s to go
on time step 821 (time=41.05), 0.00846681 s/step
Meep progress: 64.95/100.0 = 65.0% done in 12.0s, 6.5s to go
on time step 1299 (time=64.95), 0.00836969 s/step
Meep progress: 88.4/100.0 = 88.4% done in 16.0s, 2.1s to go
on time step 1769 (time=88.45), 0.00852332 s/step
run 0 finished at t = 100.0 (2000 timesteps)

📍 Far-field at [10.0, 0.0, 0.0] μm:
E_far     = [ 1.35470016e-01+6.86606701e-02j -4.42973837e-15-1.60798024e-15j
  1.91613003e+01-2.23774235e+01j]
H_meep    = [-6.14537156e-02-1.84313936e-01j -1.91615307e+01+2.23777660e+01j
 -3.11590218e-15-1.08399888e-15j]
H_calc    = [ 0.00000000e+00+0.00000000e+00j -5.08621568e-02+5.93991014e-02j
 -1.17583903e-17-4.26825642e-18j]

--- Norms ---
|E|        = 29.460611
|H_meep|   = 29.461270
|H_calc|   = 0.